In [1]:
import numpy as np
import numba as nb

In [2]:
def process_number(x):
    if x > 0:
        res = np.floor(x % 3)
    else:
        res = np.floor(x % 5)
    return res

In [3]:
process_number(0.3)

0.0

In [4]:
def process_loop(x):
    res = np.empty_like(x)
    for i in range(len(x)):
        if x[i] > 0:
            res[i] = np.floor(x[i] % 3)
        else:
            res[i] = np.floor(x[i] % 5)
    return res


def process_numpy(x):
    
    res = np.empty_like(x)

    idx = np.where(x > 0)    
    res[idx] = np.floor(x[idx] % 3)

    idx = np.where(x <= 0)
    res[idx] = np.floor(x[idx] % 5)

    return res


@nb.jit
def process_numba(x):
    res = np.empty_like(x)
    for i in range(len(x)):
        if x[i] > 0:
            res[i] = np.floor(x[i] % 3)
        else:
            res[i] = np.floor(x[i] % 5)
    return res

In [5]:
numbers = np.linspace(-100000,100000, 200000)
process_loop(numbers)[-200:-120]

array([ 2.,  0.,  1.,  2.,  0.,  1.,  2.,  0.,  1.,  2.,  0.,  1.,  2.,
        0.,  1.,  2.,  0.,  1.,  2.,  0.,  1.,  2.,  0.,  1.,  2.,  0.,
        1.,  2.,  0.,  1.,  2.,  0.,  1.,  2.,  0.,  1.,  2.,  0.,  1.,
        2.,  0.,  1.,  2.,  0.,  1.,  2.,  0.,  1.,  2.,  0.,  1.,  2.,
        0.,  1.,  2.,  0.,  1.,  2.,  0.,  1.,  2.,  0.,  1.,  2.,  0.,
        1.,  2.,  0.,  1.,  2.,  0.,  1.,  2.,  0.,  1.,  2.,  0.,  1.,
        2.,  0.])

In [6]:
%timeit process_loop(numbers)
%timeit process_numpy(numbers)
%timeit process_numba(numbers)
%timeit np.floor(np.where(numbers > 0, numbers % 3, numbers % 5))

1 loop, best of 3: 212 ms per loop
100 loops, best of 3: 4.99 ms per loop
The slowest run took 62.84 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 1.66 ms per loop
100 loops, best of 3: 6.76 ms per loop


In [7]:
@np.vectorize
def process_numpy_vec(x):
    if x > 0:
        res = np.floor(x % 3)
    else:
        res = np.floor(x % 5)
    return res


@nb.vectorize
def process_numba_vec(x):
    if x > 0:
        res = np.floor(x % 3)
    else:
        res = np.floor(x % 5)
    return res

In [8]:
%timeit process_numpy_vec(numbers)
%timeit process_numba_vec(numbers)

10 loops, best of 3: 149 ms per loop
The slowest run took 23.21 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 1.48 ms per loop


# A note on ufuncs

Numba vectorize actually creates numpy ufuncs. These are very flexible, and very powerful but can be a bit arcane.

Look up the details on ufuncs at https://docs.scipy.org/doc/numpy/reference/ufuncs.html.

Numbas documentation is here: http://numba.pydata.org/numba-doc/dev/user/vectorize.html

In [9]:
@nb.vectorize
def add(x,y):
    return x + y

# Make sure that add is compiled for floats:
add(0., 0.)

# Now we can use add.outer to create an outer product.
# Great for making distance matrices for example:
add.outer(np.linspace(0,1,5),np.linspace(2,3,5))

array([[ 2.  ,  2.25,  2.5 ,  2.75,  3.  ],
       [ 2.25,  2.5 ,  2.75,  3.  ,  3.25],
       [ 2.5 ,  2.75,  3.  ,  3.25,  3.5 ],
       [ 2.75,  3.  ,  3.25,  3.5 ,  3.75],
       [ 3.  ,  3.25,  3.5 ,  3.75,  4.  ]])